In [1]:
import pandas as pd
import ast
import os,sys
import subprocess
from datetime import datetime
import backtrader as bt

os.chdir('/Volumes/Data/Chipy/Py-Finance/Crypto/TechnicalAnalysis/')

In [2]:
class SmaCross(bt.SignalStrategy):
    params = (('pfast', 10), ('pslow', 30),('printlog',False))
    def __init__(self):
        # Keep a reference to the "close" line in the data[0] dataseries
        self.dataclose = self.datas[0].close
        
        sma1, sma2 = bt.ind.SMA(period=self.p.pfast), bt.ind.SMA(period=self.p.pslow)
        self.signal_add(bt.SIGNAL_LONGSHORT, bt.ind.CrossOver(sma1, sma2))

    def log(self, txt, dt=None, doprint=False):
        ''' Logging function fot this strategy'''
        if self.params.printlog or doprint:
            dt = dt or self.datas[0].datetime.date(0)
            print('%s, %s' % (dt.isoformat(), txt))

    def stop(self):
        test_run = {}
        test_run['pfast'] = self.params.pfast
        test_run['pslow'] = self.params.pslow
        test_run['final_value'] = (self.broker.getvalue()+self.broker.getcash())
        self.log('|{}'.format(test_run),doprint=True)
        #self.log('(pfast %2d) (pslow %2d) Ending Value %.2f' % (self.params.pfast,self.params.pslow,self.broker.getvalue()), doprint=True)

In [11]:
cerebro = bt.Cerebro()

#data = bt.feeds.YahooFinanceData(dataname='YHOO', fromdate=datetime(2011, 1, 1),todate=datetime(2012, 12, 31))
data = bt.feeds.GenericCSVData(dataname='btc_usd.csv',dtformat='%Y-%m-%d',openinterest=-1)

cerebro.adddata(data)
cerebro.addsizer(bt.sizers.PercentSizer,percents=100)

#cerebro.addstrategy(SmaCross)
strats = cerebro.optstrategy(SmaCross,pfast=range(5,12),pslow=range(15,35))

cerebro.addsizer(bt.sizers.PercentSizer,percents=100)

f = open('output_ls.txt','w')
cerebro.run()
sys.stdout = f

In [16]:
filename = 'output_ls.txt'
with open(filename) as f:
    content = f.readlines()
content = [x.split('|')[1].strip("\n") for x in content] 

In [17]:
content

["{'final_value': 102450.66063069695, 'pfast': 5, 'pslow': 16}",
 "{'final_value': 99718.57489651546, 'pfast': 5, 'pslow': 21}",
 "{'final_value': 19159.08399413529, 'pfast': 5, 'pslow': 24}",
 "{'final_value': 20509.520590526477, 'pfast': 5, 'pslow': 27}",
 "{'final_value': 20197.457882360304, 'pfast': 5, 'pslow': 31}",
 "{'final_value': 91049.27378291119, 'pfast': 6, 'pslow': 15}",
 "{'final_value': 123665.85456515528, 'pfast': 6, 'pslow': 19}",
 "{'final_value': 58748.01147764161, 'pfast': 6, 'pslow': 23}",
 "{'final_value': 61561.30906492894, 'pfast': 6, 'pslow': 27}",
 "{'final_value': 24390.344963985444, 'pfast': 6, 'pslow': 31}",
 "{'final_value': 53142.38575449237, 'pfast': 7, 'pslow': 15}",
 "{'final_value': 158331.28504066216, 'pfast': 7, 'pslow': 19}",
 "{'final_value': 66971.21353975631, 'pfast': 7, 'pslow': 23}",
 "{'final_value': 25203.90805369093, 'pfast': 7, 'pslow': 27}",
 "{'final_value': 20165.097472975165, 'pfast': 7, 'pslow': 31}",
 "{'final_value': 55337.323854659

In [18]:
run_data = pd.DataFrame([ast.literal_eval(x) for x in content])
run_data.head()

,final_value,pfast,pslow
0,102450.660631,5,16
1,99718.574897,5,21
2,19159.083994,5,24
3,20509.520591,5,27
4,20197.457882,5,31


In [19]:
run_data.sort_values('final_value',ascending=False).head(10)

,final_value,pfast,pslow
26,321602.482718,10,19
91,297849.804772,9,20
21,278940.893077,9,19
56,269547.240463,9,21
51,240664.996579,8,21
50,207853.822603,8,18
30,174748.481262,11,17
60,169920.807576,10,17
11,158331.285041,7,19
16,140319.531549,8,19
